In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

project direcotry is  /home/yuan/bio/antibody-antigen-complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/antibody-antigen-complex']


## view structure

In [3]:
# get *pdb
from src.download import Download

pdb_id = '1E4W'
pdb_file = Download.download_pdb(pdb_id)

../pdb/1E4W.pdb is ready.


In [4]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(pdb_file).read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex

In [5]:
# get chains
from src.parse_abag import ParseAbAg

outputs_dir = '../outputs'
p = ParseAbAg(pdb_file, outputs_dir)
# p.structure.header

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 1E4W
outputs dir:  ../outputs/1E/1E4W
[{'chains': [{'AA residues': 398,
              'chain_id': 'H',
              'seq': 'QVQLQQPGAELVKPGASVKLSCKASGFTFTNYWMHWVKQRPGQGLEWIGEILPSNGRTNYNEKFKTKATLTVDKSSNTAYMQLSSLTSEDSAVYYCARSPSDYWGQGTTLTVSSAKTTAPSVYPLAPVCGDTTGSSVTLGCLVKGYFPEPVTLTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVTSSTWPSQSITCNVAHPASSTKVDKKIEP'},
             {'AA residues': 434,
              'chain_id': 'L',
              'seq': 'DIQMTQTPSSLSASLGDRVTISCRASQDISHYLNWFQQKPDGTVKLLIYYTSTLHSGVPSRFSGSGSGTDYSLTISNLEEEDIAFYFCQQGGALPFTFGSGTKLAIKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLDSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC'},
             {'AA residues': 13, 'chain_id': 'P', 'seq': 'SHFNEYE'}],
  'model_id': 0}]


In [6]:
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'tab2',
  'chain': 'h',
  'fragment': 'ig kappa heavy chain',
  'other_details': 'murine fab-fragment'},
 '2': {'misc': '',
  'molecule': 'tab2',
  'chain': 'l',
  'fragment': 'ig kappa light chain',
  'other_details': 'murine fab-fragment'},
 '3': {'misc': '',
  'molecule': 'cyclic peptide',
  'chain': 'p',
  'engineered': 'yes'}}

In [7]:
# filter: is antibody-antigen complex
p.filter_antibody_antigen()
p.update_chains('chain_type')
print(p.structure_id)
pprint(p.chains[0]['chains'][0])

Pairwise chains of 1E4W are updated.
1E4W
{'AA residues': 398,
 'chain_id': 'H',
 'chain_type': 'heavy',
 'seq': 'QVQLQQPGAELVKPGASVKLSCKASGFTFTNYWMHWVKQRPGQGLEWIGEILPSNGRTNYNEKFKTKATLTVDKSSNTAYMQLSSLTSEDSAVYYCARSPSDYWGQGTTLTVSSAKTTAPSVYPLAPVCGDTTGSSVTLGCLVKGYFPEPVTLTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVTSSTWPSQSITCNVAHPASSTKVDKKIEP'}


In [8]:
# split chains
p.chains_to_pdb()

Saved chain ['H'] to ../outputs/1E/1E4W/1E4W_H.pdb
Saved chain ['P'] to ../outputs/1E/1E4W/1E4W_P.pdb
Saved chain ['H', 'P'] to ../outputs/1E/1E4W/1E4W_H_P.pdb
Saved chain ['L'] to ../outputs/1E/1E4W/1E4W_L.pdb
Saved chain ['L', 'P'] to ../outputs/1E/1E4W/1E4W_L_P.pdb


In [9]:
p.bounded

[{'type': 'heavy-antigen',
  'chains': [['H'], ['P'], ['H', 'P']],
  'pdb_files': ['../outputs/1E/1E4W/1E4W_H.pdb',
   '../outputs/1E/1E4W/1E4W_P.pdb',
   '../outputs/1E/1E4W/1E4W_H_P.pdb']},
 {'type': 'light-antigen',
  'chains': [['L'], ['P'], ['L', 'P']],
  'pdb_files': ['../outputs/1E/1E4W/1E4W_L.pdb',
   '../outputs/1E/1E4W/1E4W_P.pdb',
   '../outputs/1E/1E4W/1E4W_L_P.pdb']}]

In [10]:
p.save_annot()

https://www.ebi.ac.uk/pdbe/api/mappings/1e4w
Save annotation of 1E4W: ../outputs/1E/1E4W/meta.json


## SASA

In [11]:
# calculate SASA
sasa_file = p.build_freesasa()

In [12]:
from src.analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa(sasa_file)
p2.print_meta()
print(list(p2.data[0]['delta_total_sasa']))

key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
heavy chain H ~ antigen chain P
------------------------------
key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
light chain L ~ antigen chain P
------------------------------
['delta1', 'delta1_file', 'delta2', 'delta2_file']


In [13]:
# retrieve sequences
motifs = p2.retrieve_seq()
print('output file: ', motifs[1])
motifs

output file:  ../outputs/1E/1E4W/seq_sasa_delta-0_span-30.p


([('heavy',
   [{'seq': 'WMHWVKQRPGQGLEWIGEILPSNGRTN',
     'sig_res': 4,
     'start': (32, '33'),
     'end': (58, '58'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'SPSD',
     'sig_res': 4,
     'start': (98, '95'),
     'end': (101, '98'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}],
   'antigen',
   [{'seq': 'SHFNEYE',
     'sig_res': 6,
     'start': (0, '1'),
     'end': (6, '7'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}]),
  ('light',
   [{'seq': 'YLNWFQQKPDGTVKLLIYY',
     'sig_res': 6,
     'start': (31, '32'),
     'end': (49, '50'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'QQGGALPF',
     'sig_res': 6,
     'start': (88, '89'),
     'end': (95, '96'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}],
   'antigen',
   [{'seq': 'SHFNEYE',
     'sig_res': 6,
     'start': (0, '1'),
     'end': (6, '7'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}])],
 '../outputs/1E/1E4W/seq_sasa_delta-0_span-30.p')

In [14]:
# dataset for ML
seq_sasa_file = motifs[1]
AnalyzeSasa(seq_sasa_file).build_dataset()

[{'input': 'SHFNEYE',
  'input_label': 'antigen',
  'output': 'WMHWVKQRPGQGLEWIGEILPSNGRTN | SPSD',
  'output_label': 'heavy'},
 {'input': 'SHFNEYE',
  'input_label': 'antigen',
  'output': 'YLNWFQQKPDGTVKLLIYY | QQGGALPF',
  'output_label': 'light'}]

In [15]:
# complex
p2.print_delta_sasa(0)

complex: heavy chain H ~ antigen chain P
------------------------------
heavy chain:
    chain_id res_no  res aa      value
32         H     33  TRP  W  44.839223
34         H     35  HIS  H  20.077240
49         H     50  GLU  E  15.332530
58         H     58  ASN  N   9.290670
98         H     95  SER  S  19.362616
99         H     96  PRO  P  62.189259
100        H     97  SER  S   1.161334
101        H     98  ASP  D  30.604403
------------------------------
antigen chain:
  chain_id res_no  res aa      value
0        P      1  SER  S  28.232312
1        P      2  HIS  H  49.216165
3        P      4  ASN  N  68.601878
4        P      5  GLU  E  61.111643
5        P      6  TYR  Y  12.991265
6        P      7  GLU  E   6.759702


## distance

In [16]:
# calculate distance
dist_file = p.build_dist()

In [17]:
from src.analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()
print(list(p3.data[0]['dist']))

key1 ['type', 'chains', 'pdb_files', 'dist']
heavy chain H ~ antigen chain P
------------------------------
key1 ['type', 'chains', 'pdb_files', 'dist']
light chain L ~ antigen chain P
------------------------------
['dist1', 'dist1_file', 'dist2', 'dist2_file']


In [18]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs

([('heavy',
   [{'seq': 'WMHWVKQRPGQGLEWIGEILSNGRTN',
     'sig_res': 4,
     'start': (32, 33),
     'end': (57, 58),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'SPSD',
     'sig_res': 4,
     'start': (94, 95),
     'end': (97, 98),
     'dist_cutoff': 5,
     'max_span': 30}],
   'antigen',
   [{'seq': 'SHFNEYE',
     'sig_res': 6,
     'start': (0, 1),
     'end': (6, 7),
     'dist_cutoff': 5,
     'max_span': 30}]),
  ('light',
   [{'seq': 'YLNWFQQKPDGTVKLLIYY',
     'sig_res': 5,
     'start': (31, 32),
     'end': (49, 50),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'QQGGALPF',
     'sig_res': 7,
     'start': (88, 89),
     'end': (95, 96),
     'dist_cutoff': 5,
     'max_span': 30}],
   'antigen',
   [{'seq': 'SHFNEYE',
     'sig_res': 5,
     'start': (0, 1),
     'end': (6, 7),
     'dist_cutoff': 5,
     'max_span': 30}])],
 '../outputs/1E/1E4W/seq_dist_dist-5_span-30.p')

In [19]:
seq_dist_file = motifs[1]
AnalyzeDist(seq_dist_file).build_dataset()

[{'input': 'SHFNEYE',
  'input_label': 'antigen',
  'output': 'WMHWVKQRPGQGLEWIGEILSNGRTN | SPSD',
  'output_label': 'heavy'},
 {'input': 'SHFNEYE',
  'input_label': 'antigen',
  'output': 'YLNWFQQKPDGTVKLLIYY | QQGGALPF',
  'output_label': 'light'}]

In [20]:
# complex
p3.print_min_dist(0)

------------------------------
     res_no  res atom      value aa
0         1  GLN    C  16.685555  Q
1         2  VAL    C  12.640076  V
2         3  GLN    C  15.828628  Q
3         4  LEU    C  11.713461  L
4         5  GLN    C  17.599824  Q
..      ...  ...  ...        ... ..
204     205  LYS    C  49.531600  K
205     206  LYS    C  52.087440  K
206     207  ILE    C  49.886531  I
207     208  GLU    C  54.913281  E
208     209  PRO    C  57.663678  P

[209 rows x 5 columns]
------------------------------
   res_no  res atom     value aa
0       1  SER    C  2.621994  S
1       2  HIS    C  3.309439  H
3       4  ASN    C  2.825416  N
4       5  GLU    C  2.540054  E
5       6  TYR    C  3.619626  Y
6       7  GLU    C  3.954977  E
